# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for py4j==0.10.9.7 from https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 4.9 MB/s eta 0:00:00ta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840633 sha256=cd47b2ac374164f45bd6cf4a2a7eafb736669a2830c1cc4021bdfc9e57281093
  Stored in directory: /Users/mac/Library/Caches/pip/wheels/97/f5/c0/947e2c0942b361ffe58651f36bd7f13772675b3863fd63d1b1
Successfully built pyspark


In [61]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, lower, row_number, collect_list, struct, sort_array, expr, concat_ws
from pyspark.sql.window import Window
import json

In [5]:
# Initialize Spark session in local mode
spark = SparkSession.builder \
    .appName("ClickstreamRouteAnalysis") \
    .master("local[2]") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

24/10/28 02:50:51 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [49]:
file_path = "clickstream.csv"
clickstream_df = spark.read.option("header", "true").csv(file_path)

In [50]:
clickstream_df.printSchema()

root
 |-- user_id\tsession_id\tevent_type\tevent_page\ttimestamp: string (nullable = true)



In [51]:
# Handle cases where the CSV is not properly parsed
if len(clickstream_df.columns) == 1:
    clickstream_df = clickstream_df.withColumn(
        "user_id", split(col(clickstream_df.columns[0]), "\t")[0].cast("int")
    ).withColumn(
        "session_id", split(col(clickstream_df.columns[0]), "\t")[1].cast("int")
    ).withColumn(
        "event_type", split(col(clickstream_df.columns[0]), "\t")[2]
    ).withColumn(
        "page", split(col(clickstream_df.columns[0]), "\t")[3]
    ).withColumn(
        "timestamp", split(col(clickstream_df.columns[0]), "\t")[4].cast("long")
    ).select("user_id", "session_id", "event_type", "page", "timestamp")
else:
    clickstream_df = clickstream_df.select(
        col('user_id').cast('int'),
        col('session_id').cast('int'),
        col('event_type'),
        col('page'),
        col('timestamp').cast('long')
    )

In [74]:
clickstream_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- session_id: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- page: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- row_num: integer (nullable = false)
 |-- error_row_num: integer (nullable = true)



In [54]:
# Convert event_type to lowercase for error detection
clickstream_df = clickstream_df.withColumn('event_type', lower(col('event_type')))

# Define window specification
window_spec = Window.partitionBy('user_id', 'session_id').orderBy('timestamp')

# Add row numbers within each session
clickstream_df = clickstream_df.withColumn('row_num', row_number().over(window_spec))

In [55]:
# Identify the first error occurrence in each session
error_df = clickstream_df.filter(col('event_type').contains('error')) \
    .select('user_id', 'session_id', 'row_num') \
    .groupBy('user_id', 'session_id') \
    .agg({'row_num': 'min'}) \
    .withColumnRenamed('min(row_num)', 'error_row_num')

In [56]:
# Join back to the main DataFrame
clickstream_df = clickstream_df.join(error_df, on=['user_id', 'session_id'], how='left')


In [59]:
# Filter out events after the error
clickstream_df = clickstream_df.filter(
    (col('error_row_num').isNull()) | (col('row_num') < col('error_row_num'))
)

# Select page events
clickstream_pages = clickstream_df.filter(col('event_type') == 'page')

# Create a struct of (row_num, page) to maintain order
clickstream_pages = clickstream_pages.withColumn(
    'page_struct', struct(col('row_num'), col('page'))
)

In [62]:
# Group by user_id and session_id, collect and sort pages
routes_df = clickstream_pages.groupBy('user_id', 'session_id') \
    .agg(collect_list('page_struct').alias('pages_struct')) \
    .withColumn('sorted_pages_struct', sort_array(col('pages_struct'))) \
    .withColumn('pages', expr("transform(sorted_pages_struct, x -> x.page)"))


In [69]:
routes_df = routes_df.withColumn('route', concat_ws('-', col('pages')))

# Count routes and order by frequency
route_counts = routes_df.groupBy('route').count().orderBy(col('count').desc())

# Get the top 30 routes
top_routes = route_counts.limit(30)

In [72]:
# Save the top 30 routes to a CSV file
top_routes.select('route', 'count').write.csv('top_routes.csv', header=True)

In [73]:
# Save the top 10 routes to result.json
top_10_routes = top_routes.limit(10).collect()
result_dict = {row['route']: row['count'] for row in top_10_routes}

with open('result.json', 'w') as f:
    json.dump(result_dict, f)

In [ ]:
# Stop the Spark session
spark.stop()